In [ ]:
# %%time
import torch
import numpy as np
from scipy.optimize import minimize, Bounds
import cProfile, io, pstats
from pstats import SortKey
from botorch.models.gp_regression import SingleTaskGP
from botorch.acquisition import qNoisyExpectedImprovement

torch.manual_seed(0)
np.random.seed(0)

torch.set_default_dtype(torch.double)

train_X = torch.rand(256, 2)
train_Y = torch.sin(train_X * 10).sum(dim=-1, keepdim=True)

model = SingleTaskGP(train_X=train_X, train_Y=train_Y)
acqf = qNoisyExpectedImprovement(model, train_X, cache_root=False)

# rand_tensor = torch.rand(1024, 128, 128, dtype=torch.double)

def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = acqf(X_tensor).sum()
#     diff = rand_tensor - X_tensor
#     loss = (diff @ diff).sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

bounds = Bounds(
    lb=np.zeros(128),
    ub=np.ones(128),
    keep_feasible=True,
)
x0 = np.random.random(128)

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"ftol": 0, "gtol": 0, "maxiter": 20},
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats()
None

In [ ]:
def f_and_grad(x):
    X_tensor = torch.from_numpy(x.reshape((16, 4, 2))).contiguous().requires_grad_(True)
    loss = acqf._get_samples_and_objectives(X_tensor)[1].sum()
    grad = torch.autograd.grad(loss, X_tensor)[0].reshape(-1).numpy(force=True)
    return loss.item(), grad

with cProfile.Profile() as pr:
    res = minimize(
        fun=f_and_grad,
        x0=x0,
        method="L-BFGS-B",
        jac=True,
        bounds=bounds,
        options={"ftol": 0, "gtol": 0, "maxiter": 20},
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats()
None